### Import the necessary documents 

In [83]:
# file management stuff
from tkinter import Tk, filedialog
import os

import re
import pandas as pd

# doc parsing
import win32com.client

import random

# pdf parsing
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io
from io import StringIO

# html parsing
import codecs
from bs4 import BeautifulSoup

import csv

# to run parsing in parallel
import concurrent.futures

# from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool


from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
porter=PorterStemmer()

import numpy as np
from numpy import nan

### Select the folder holding all of the syllabi 

In [2]:
root = Tk()
root.directory = filedialog.askdirectory()
directory = root.directory
root.withdraw() #hide the main window 

''

### Select a document if you dare 

In [54]:
root.filename = filedialog.askopenfilename()
file = root.filename
root.withdraw()

''

In [43]:
stopwords = []
with open('stopwords_jul.txt', 'r') as f:
    for line in f:
        stopwords.append(line.strip())

In [29]:
bad_course = ['W16-OSPFLOR-69-01.doc', 'F13-RELIGST-55-01.doc', 'W09-BIO-213-01.doc']

In [30]:
course_list_doc = []
for filename in os.listdir(directory):
    if filename in bad_course:
        continue
    if '.doc' in filename:
        course_list_doc.append(filename)

In [31]:
syllabi_df = {'CLASS': [], 'syllabi': [], 'website_count': []}

In [32]:
len(course_list_doc)

16717

In [33]:
for i in range(4):
    random.shuffle(course_list_doc)

In [34]:
course_list = course_list_doc[:500]

In [51]:
loop = 0
for file in course_list:

    text, count = parser(file)
    choice = re.search('[a-zA-Z]', text)
    
    if choice == None:
        continue
        
    syllabi_df['CLASS'].append(file)
    syllabi_df['syllabi'].append(text)
    syllabi_df['website_count'].append(count)
    loop +=1
    
    print(loop)
    print(file)

1
Sp10-HISTORY-50C-01.doc
2
Sp09-AMELANG-129C-01.doc
3
Sp11-ENGLISH-94-02.doc
4
W11-IHUM-11A-04.docx
5
Sp14-GES-184-01.docx
6
W14-ECON-179-01.doc
7
W11-PORTLANG-1A-01.doc
8
W16-FEMGEN-212X-01.docx
9
Sp10-SOC-209-01.doc
10
W09-CHINLANG-2-01.doc
11
Sp13-IPS-201-01.docx
12
W09-IHUM-40A-09.doc
13
Sp09-IHUM-6B-09.doc
14
F11-LINGUIST-250-01.docx
15
Sp10-HISTORY-209S-01.doc
16
F14-MS&E-388-01.docx
17
W13-SOC-146-01.doc
18
F15-CHEM-35-28.doc
19
F09-LINGUIST-5N-01.doc
20
W16-COMPLIT-368A-01.doc
21
W14-THINK-21-03.doc
22
W12-LAW-656-01.doc
23
F11-HRP-283-01.doc
24
W15-EDUC-245-01.docx
25
Sp11-CSRE-173S-01.doc
26
W12-EDUC-316-01.doc
27
W14-PUBLPOL-204-01.docx
28
F14-ARTSTUDI-160-01.doc
29
Sp11-SOC-359-01.docx
30
F09-PORTLANG-1A-03.doc
31
F10-GENE-272-01.doc
32
W09-IHUM-4A-01.doc
33
W10-URBANST-110-01.doc
34
W14-ILAC-130-01.docx
35
Sp09-SPANLIT-106N-01.doc
36
W13-PSYCH-147-01.doc
37
F09-AMELANG-128A-01.doc
38
F14-EFSLANG-698A-02.docx
39
Sp14-MED-157-01.doc
40
Sp12-IHUM-69B-14.doc
41
W16-ANTHRO-301

316
W12-HUMBIO-129-01.doc
317
W14-ARTHIST-287-01.doc
318
Sp11-ENGLISH-141A-01.doc
319
Sp09-GEOPHYS-101-01.doc
320
W10-ARTHIST-132-01.docx
321
W16-AFRICAAM-150B-01.docx
322
Sp09-BIO-158-01.doc
323
W10-EFSLANG-691-01.doc
324
F13-GES-183-01.doc
325
F08-CHINLANG-395-09.doc
326
F16-EFSLANG-693A-01.doc
327
F10-ITALGEN-155-01.doc
328
F10-EDUC-220C-02.doc
329
F12-POLISCI-334-01.docx
330
W09-MED-242-01.doc
331
Sp15-CSRE-165-01.docx
332
Sp08-SPANLIT-278A-01.doc
333
W10-GERLANG-100-01.doc
334
F15-LAW-715B-01.docx
335
Sp14-LAW-224-10.doc
336
F15-FRENLANG-21C-03.doc
337
F10-LAW-274-01.docx
338
W16-POLISCI-224C-01.docx
339
W12-BIO-160B-01.doc
340
F13-SPECLANG-178A-02.docx
341
W15-CHINLANG-1-01.doc
342
F14-HISTORY-115D-01.doc
343
F12-ITALLANG-21A-01.doc
344
Sp14-HISTORY-203-01.docx
345
Sp13-ILAC-110-01.docx
346
F15-ANTHRO-91A-01.docx
347
Sp16-ITALLANG-2A-01.doc
348
F08-IHUM-66-16.doc
349
Sp14-SPANLANG-15-01.doc
350
W15-TAPS-14N-01.docx
351
Sp12-PSYCH-147-01.doc
352
Sp13-CSRE-279G-01.docx
353
F08-SOC-

In [52]:
for i in range(len(syllabi_df['CLASS'])):
    item = syllabi_df['CLASS'][i]
    
    beginning= item.find('-')
    end = item.rfind('-')
    new = item[beginning + 1 : end]
    
    syllabi_df['CLASS'][i] = new
syllabi_df = pd.DataFrame(syllabi_df)

In [56]:
game_count = 0
main_dict = {'SUBJECT': [], 'CLASS': [], 'CATALOG_NBR': [], 'CATALOG_DESCRIPTION': []}
with open(file, newline = '', encoding='utf-8') as games:                                                                                          
    game_reader = csv.reader(games, delimiter='\t')
    for game in game_reader:
        game_count+= 1
        if game_count >= 2:
            if game[4] == 'NULL' or game[4] == 'TBA':
                continue
            
            text = re.sub("<!--?.*?-->","",text)
            text = re.sub("(\\d|\\W)+"," ", text)
            text = removeStopWords(text)
            text = game[4].lower()
            text = stemSentence(text)            
            
            main_dict['SUBJECT'].append(game[0])
            main_dict['CLASS'].append(game[0]+ '-' + game[1])
            main_dict['CATALOG_NBR'].append(game[1])
            main_dict['CATALOG_DESCRIPTION'].append(text)
descr_df = pd.DataFrame(main_dict)

In [57]:
# drop empty course descriptions

indexNamesDescr = descr_df[descr_df['CATALOG_DESCRIPTION'] == '' ].index
indexNamesDescr2 = descr_df[descr_df['CATALOG_DESCRIPTION'] == ' ' ].index
descr_df.drop(indexNamesDescr , inplace=True)
descr_df.drop(indexNamesDescr2 , inplace=True)

In [62]:
merged_inner = pd.merge(left=descr_df, right=syllabi_df, left_on='CLASS', right_on='CLASS')
left_merge = syllabi_df.merge(descr_df, on='CLASS', how='left')

In [86]:
descr = left_merge['CATALOG_DESCRIPTION'].tolist()
syll = left_merge['syllabi'].tolist()
jacc_sim = []
for i in range(len(descr)):
    if descr[i] is np.nan:
        jacc_sim.append(0)
        continue
    description = descr[i].split()
    syllabus = syll[i].split()
    jac_ind = jaccard_similarity(description, syllabus)
    jacc_sim.append(jac_ind)

left_merge['jaccard_similarity'] = jacc_sim

In [93]:
descr = merged_inner['CATALOG_DESCRIPTION'].tolist()
syll = merged_inner['syllabi'].tolist()
jacc_sim = []
for i in range(len(descr)):
    description = descr[i].split()
    syllabus = syll[i].split()
    jac_ind = jaccard_similarity(description, syllabus)
    jacc_sim.append(jac_ind)

merged_inner['jaccard_similarity'] = jacc_sim

In [89]:
final_df = merged_inner.sort_values(by='jaccard_similarity', ascending=False).reset_index(drop=True)
left_final = left_merge.sort_values(by='jaccard_similarity', ascending=False).reset_index(drop=True)

In [91]:
left_final

,CLASS,syllabi,website_count,SUBJECT,CATALOG_NBR,CATALOG_DESCRIPTION,jaccard_similarity
0,CEE-174B,cee winter wastewat treatment dispos resourc r...,0,CEE,174B,"thi cours build upon cee 174a , cover basic hy...",0.156134
1,ENGLISH-134,marriag plot seminar investig central marriag ...,0,ENGLISH,134,the central of the marriag plot in the develop...,0.134529
2,COMPMED-87Q,compm introduct mous biomed research autumn lk...,0,COMPMED,87Q,prefer to sophomor . thi cours focus on the la...,0.094203
3,COMPMED-87Q,compm introduct mous biomed research syllabu l...,0,COMPMED,87Q,prefer to sophomor . thi cours focus on the la...,0.093373
4,BIO-110,fall quarter tth lathrop chromatin regul genom...,0,BIO,110,mainten of the genom is a prerequisit for life...,0.089588
...,...,...,...,...,...,...,...
540,HISTORY-203,hist premodern econom cultur tuth pm pm prof l...,4,NaN,NaN,NaN,0.000000
541,ILAC-110,la sociedad española del siglo xxi travé del c...,3,NaN,NaN,NaN,0.000000
542,ILAC-349,literari theori héctor hoyo nariman skakov sta...,1,NaN,NaN,NaN,0.000000
543,IHUM-69B,stanford univers human histori ii ihum spring ...,2,NaN,NaN,NaN,0.000000


In [61]:
# make sure that both description and syllabi are lowercase CHECK
# lammetize (porter stemmer in nltk) to chop off word endings to get root word *** CHECK
# remove stop words CHECK
# keep repeated course offerings so do a left join and drop instances where its null in course description
# vizualizaztions of how similarity varies across departments
# explore website linking. See if the number of websites have decreased or increased in frequency over time
# add each aspect of filename to dataframe

final_df

,SUBJECT,CLASS,CATALOG_NBR,CATALOG_DESCRIPTION,syllabi,website_count,jaccard_similarity
0,CEE,CEE-174B,174B,"thi cours build upon cee 174a , cover basic hy...",cee winter wastewat treatment dispos resourc r...,0,0.156134
1,ENGLISH,ENGLISH-134,134,the central of the marriag plot in the develop...,marriag plot seminar investig central marriag ...,0,0.134529
2,COMPMED,COMPMED-87Q,87Q,prefer to sophomor . thi cours focus on the la...,compm introduct mous biomed research autumn lk...,0,0.094203
3,COMPMED,COMPMED-87Q,87Q,prefer to sophomor . thi cours focus on the la...,compm introduct mous biomed research syllabu l...,0,0.093373
4,BIO,BIO-110,110,mainten of the genom is a prerequisit for life...,fall quarter tth lathrop chromatin regul genom...,0,0.089588
...,...,...,...,...,...,...,...
325,CHINLANG,CHINLANG-395,395,prerequisit : consent of instructor .,modern chines stanford fall instructor hong ze...,6,0.000948
326,PORTLANG,PORTLANG-99,99,prerequisit : consent of instructor.n ( staff ),spring portugues syllabu stanford univers inst...,0,0.000000
327,MUSIC,MUSIC-41,41,pre- or corequisit : 22 .,music histori western art music professor heat...,0,0.000000
328,MUSIC,MUSIC-40,40,pre- or corequisit : 21 .,music western music histori stanford univers f...,0,0.000000


In [50]:
def removeStopWords(string):
    word_list = string.split()
    new_list = []
    for word in word_list:
        if word not in stopwords:
            new_list.append(word)
    return " ".join(new_list)

In [17]:
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [18]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [19]:
def find_url(string): 
  
    # findall() has been used  
    # with valid conditions for urls in string 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)       
    return [x[0] for x in url] 

In [20]:
def parser(file):
    path = directory + '/' + file
    if '.pdf' in file:
        text, count = convert_pdf_to_txt_2(path)
    elif '.doc' in file:
        text, count = doc_parser(path)
    else:
        text, count = htm_parser(path)
    return text, count

In [46]:
def convert_pdf_to_txt_2(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)
        
    fp.close()
    device.close()
    text = retstr.getvalue()
    text_vec = text.split('\n')
    text_total =  ' '
    for word in text_vec:
        text_total += ' ' + word
    retstr.close()
    
    
    url_list = find_url(text_total)
    email_list = re.findall('\S+@\S+', text_total)
    
    for url in url_list:
        text_total = text_total.replace(url, "")
    
    for email in email_list:
        text_total = text_total.replace(email, "")
    
    
     # lowercase
    text_total=text_total.lower()
    
    #remove tags
    text_total=re.sub("<!--?.*?-->","",text_total)
    
    # remove special characters and digits
    
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+"," ", text_total)
    text = removeStopWords(text)
    text = stemSentence(text)
    
    
    return text, len(url_list)

In [47]:
def htm_parser(path):
    f=codecs.open(path, 'r', encoding='latin-1')
    unsoup = f.read()
    soup = BeautifulSoup(unsoup)
    for script in soup(["script", "style"]):
        script.decompose()
    strips = list(soup.stripped_strings)
    text_total= ''
    for strip in strips:
        if (strip != 'Ê') and('Ê' not in strip):
            text_total += ' ' + strip.strip('\n')
    
    
    url_list = find_url(text_total)
    email_list = re.findall('\S+@\S+', text_total)
    
    for url in url_list:
        text_total = text_total.replace(url, "")
    
    for email in email_list:
        text_total = text_total.replace(email, "")
    
    
    # lowercase
    text_total=text_total.lower()
    
    #remove tags
    text_total=re.sub("<!--?.*?-->","",text_total)
    
    # remove special characters and digits
    
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+"," ", text_total)
    text = removeStopWords(text)
    text = stemSentence(text)
    
    return text, len(url_list)

In [48]:
def doc_parser(path):
    doc = win32com.client.GetObject(path)
    text= doc.Range().Text.split('\r')
    
    text_total = ' '
    for word in text:
        if '' in word:
            word = word.replace('', '')
        if word != '':
            text_total += ' ' + word 
    
    
    url_list = find_url(text_total)
    email_list = re.findall('\S+@\S+', text_total)
    
    for url in url_list:
        text_total = text_total.replace(url, "")
    
    for email in email_list:
        text_total = text_total.replace(email, "")
    
     # lowercase
    text_total=text_total.lower()
    
    #remove tags
    text_total=re.sub("<!--?.*?-->","",text_total)
    
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+"," ", text_total)
    text = removeStopWords(text)
    text = stemSentence(text)
    
    return text, len(url_list)